In [42]:
import pandas as pd
import numpy as np
import nltk

In [43]:
DIR = 'C:\\Users\\jesse\\Desktop\\Honors Project\\'

In [44]:
emotions = pd.read_csv(DIR + 'emotions.csv')
emotions

,emotion,word,verb_form,noun_form,adjective_form,adverb_form,present_participle,past_tense,present_tense
0,Joyful,Delighted,delight,delight,delighted,delightfully,delighting,delighted,delights
1,Joyful,Happy,NaN,happiness,happy,happily,NaN,NaN,NaN
2,Joyful,Blissful,bliss,bliss,blissful,blissfully,blissing,blissed,blisses
3,Joyful,Content,content,contentment,content,contentedly,contenting,contented,contents
4,Joyful,Cheerful,cheer,cheer,cheerful,cheerfully,cheering,cheered,cheers
...,...,...,...,...,...,...,...,...,...
107,Nostalgic,Nostalgic,reminisce,nostalgia,nostalgic,nostalgically,reminiscing,reminisced,reminisces
108,Dissapointed,Dissapointed,disappoint,disappointment,dissapointed,dissapointedly,disappointing,disappointed,disappoints
109,Uncomfortable,Uncomfortable,discomfort,discomfort,uncomfortable,uncomfortably,discomforting,discomforted,discomforts
110,Offended,Offended,offend,offense,offended,offendedly,offending,offensive,offends


In [45]:
emotions['word'] = emotions['word'].str.lower()
emotions['emotion'] = emotions['emotion'].str.lower()

In [46]:
emotions = emotions.melt(id_vars=['emotion', 'word'], var_name='word_form', value_name='form')

In [47]:
emotions

,emotion,word,word_form,form
0,joyful,delighted,verb_form,delight
1,joyful,happy,verb_form,NaN
2,joyful,blissful,verb_form,bliss
3,joyful,content,verb_form,content
4,joyful,cheerful,verb_form,cheer
...,...,...,...,...
779,nostalgic,nostalgic,present_tense,reminisces
780,dissapointed,dissapointed,present_tense,disappoints
781,uncomfortable,uncomfortable,present_tense,discomforts
782,offended,offended,present_tense,offends


In [48]:
emotions = emotions.drop_duplicates(subset=['form'])
emotions = emotions.dropna(subset=['form']).reset_index(drop=True)

In [49]:
emotions = emotions.drop(columns=['word', 'word_form'])

In [50]:
emotions

,emotion,form
0,joyful,delight
1,joyful,bliss
2,joyful,content
3,joyful,cheer
4,joyful,merry
...,...,...
637,anxious,distresses
638,nostalgic,reminisces
639,dissapointed,disappoints
640,offended,offends


In [51]:
from nltk.corpus import wordnet as wn

def get_synonyms(word):
    synonyms = []
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def get_similar_words(word):
    similar_words = []
    for synset in wn.synsets(word):
        for hypernym in synset.hypernyms():
            similar_words.extend(hypernym.lemma_names())
        for hyponym in synset.hyponyms():
            similar_words.extend(hyponym.lemma_names())
    return similar_words

In [52]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def get_lexicon(word):
    lexicon = pd.DataFrame(columns=['word', 'lex_word', 'association'])
    synonyms = get_synonyms(word)
    for synonym in synonyms:
        lexicon = pd.concat([lexicon, pd.DataFrame([[word, synonym, 'syn']], columns=lexicon.columns)], ignore_index=True)
    # similar_words = get_similar_words(word)
    # for similar_word in similar_words:
    #     lexicon = pd.concat([lexicon, pd.DataFrame([[word, similar_word, 'sim']], columns=lexicon.columns)], ignore_index=True)
    #     synonyms = get_synonyms(similar_word)
    #     for synonym in synonyms:
    #         lexicon = pd.concat([lexicon, pd.DataFrame([[word, synonym, 'sim_syn']], columns=lexicon.columns)], ignore_index=True)
    # lexicon['word'] = lexicon['word'].apply(lambda x: lemmatizer.lemmatize(x))  
    lexicon = lexicon.drop_duplicates(subset=['word', 'lex_word'], keep='first')
    return lexicon

In [53]:
emotion_lex = pd.DataFrame(columns=['word', 'lex_word','association'])

In [54]:
for word in emotions['form']:
    emotion_lex = pd.concat([emotion_lex, get_lexicon(word)], ignore_index=True)
emotion_lex = emotion_lex.merge(emotions, left_on='word', right_on='form', how='left')
emotion_lex = emotion_lex.drop(columns=['form'])

In [55]:
emotion_lex_complete = emotion_lex
emotion_lex_complete

,word,lex_word,association,emotion
0,delight,delight,syn,joyful
1,delight,delectation,syn,joyful
2,delight,joy,syn,joyful
3,delight,pleasure,syn,joyful
4,delight,please,syn,joyful
...,...,...,...,...
4717,confuses,jumble,syn,confused
4718,confuses,mix_up,syn,confused
4719,confuses,blur,syn,confused
4720,confuses,obscure,syn,confused


ABOVE IS CLEAN

In [56]:
emotion_lex = emotion_lex.drop(columns=['word'])

In [60]:
emotion_lex

,lex_word,association,emotion
0,delight,syn,joyful
3136,urge,syn,inspired
3152,carnalize,syn,sensual
3151,sensualize,syn,sensual
3150,sympathize_with,syn,compassionate
...,...,...,...
1565,bucked_up,syn,inspired
1558,motivated,syn,inspired
1566,encouraged,syn,inspired
1579,focussed,syn,inspired


In [58]:
emotion_lex = emotion_lex.sort_values('association',ascending = False)

In [59]:
emotion_lex = emotion_lex.drop_duplicates(subset=['lex_word','emotion'], keep='first')

In [66]:
counts = emotion_lex['lex_word'].value_counts()
words_with_multiple_occurrences = counts[counts > 1].index
multiple = emotion_lex[emotion_lex['lex_word'].isin(words_with_multiple_occurrences)]

In [68]:
#export emotion_lex to csv
emotion_lex.to_csv(DIR + 'emotion_lex.csv', index=False)

In [ ]:
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
# from pyinflect import getAllInflections

# def get_pos(word):
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": "a",
#                 "N": "n",
#                 "V": "v",
#                 "R": "r"}  # 'R' is adverb
#     return tag_dict.get(tag, "n")

# pos = ['a', 'n', 'v', 'r']
# def get_lemms(word):
#     lemms = []
#     lemmatizer = WordNetLemmatizer()
#     for p in pos:
#         lemm = lemmatizer.lemmatize(word, p)
#         if lemm != word:
#             lemms.append(lemm)
#     return lemms

# def get_lemm(word, pos):
#     lemmatizer = WordNetLemmatizer()
#     return lemmatizer.lemmatize(word, pos)

# def get_versions(word, pos):
#     pos_dict = {"a": "A",
#                 "n": "N",
#                 "v": "V",
#                 "r": "A"}  # 'r' is adverb, but pyinflect doesn't support adverbs, so we'll treat them as adjectives
#     pos = pos_dict.get(pos, "N")
#     inflections = getAllInflections(word, pos)
#     inflections_list = [inflected_word for inflected_words in inflections.values() for inflected_word in inflected_words]
#     return inflections_list

# def get_all(word):
#     pos = get_pos(word)
#     lemm = get_lemm(word, pos)
#     if lemm is None:
#         return get_versions(word, pos)
#     else:
#         return get_versions(lemm, pos)